### Lab 06: Bernstein–Vazirani Algorithm – Finding the Secret String

In the Bernstein–Vazirani algorithm, the task is to find a hidden bitstring  ![formula](https://latex.codecogs.com/png.image?\dpi{110}&space;s\in\{0,1\}^n) such that: ![formula](https://latex.codecogs.com/png.image?\dpi{110}&space;f(x)=s\cdot%20x\pmod%202) where the dot denotes the bitwise dot product modulo 2.

For example, with  
![formula](https://latex.codecogs.com/png.image?\dpi{110}&space;n=4) and ![formula](https://latex.codecogs.com/png.image?\dpi{110}&space;s=1011): ![formula](https://latex.codecogs.com/png.image?\dpi{110}&space;f(x_0,x_1,x_2,x_3)=x_0\oplus%20x_2\oplus%20x_3)  

The function should be implemented as a **quantum oracle**. The oracle flips the target qubit for each position *i* where ![formula](https://latex.codecogs.com/png.image?\dpi{110}&space;s_i=1), using a CNOT from input qubit ![formula](https://latex.codecogs.com/png.image?\dpi{110}&space;x_i) to the target qubit.

---

#### Problem Overview

- Given a user‑specified hidden bitstring ![formula](https://latex.codecogs.com/png.image?\dpi{110}&space;s), the algorithm should recover it in a single query to the oracle.

- **Expected output:**
  - The measurement result of the input qubits will exactly match the secret string ![formula](https://latex.codecogs.com/png.image?\dpi{110}&space;s).
  - The target qubit is not measured and remains unused in the final result.

---

In [ ]:
# enable inline display of matplotlib plots in Jupyter notebooks
%matplotlib inline

from qiskit import QuantumCircuit, QuantumRegister
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram,plot_distribution
from qiskit.visualization import circuit_drawer

def bv_oracle(s):
    """
    Bernstein–Vazirani oracle for secret string s.

    The oracle implements:
        f(x) = s · x
    where '·' is the bitwise dot product modulo 2.

    For each bit s_i = 1 in the secret string:
      - A CNOT gate is applied from input qubit x_i to the target qubit.
      - This flips the target qubit if and only if x_i = 1.

    As a result, the target qubit is flipped for every input qubit position
    where s_i = 1, effectively computing the XOR of the selected input bits.
    """
    n = len(s)
    qc = QuantumCircuit(n+1)

    qc.barrier()  # visual separation from previous circuit steps
    for i, bit in enumerate(s):
        if bit == '1':
            qc.cx(i, n)  # XOR xi into target if si = 1
    
    qc.barrier()  # visual separation from previous circuit steps
    
    return qc

def bv_circuit(s):
    """
    Builds the Bernstein–Vazirani circuit for the given secret bitstring s.

    The circuit prepares the target (auxiliary) qubit in the |−⟩ state to enable
    bit flipping via the oracle. It sandwiches the oracle between two layers of
    Hadamard gates on the input qubits, and finally measures the input register
    to reveal the hidden string in a single query.
    """
    n = len(s)
    qc = QuantumCircuit(n+1, n)

    # Step 1: Initialize target qubit in |-> state
    qc.x(n)
    qc.h(n)

    # Step 2: Apply Hadamard to all input qubits
    qc.h(range(n))

    # Step 3: Apply oracle
    qc.compose(bv_oracle(s), inplace=True)
    #qc.append(bv_oracle(s), range(n+1))

    # Step 4: Apply Hadamard to input qubits again
    qc.h(range(n))

    # Step 5: Measure input qubits (ignore target)
    qc.measure(range(n), range(n))

    return qc

# ------------------------------------------------
#                main program
# ------------------------------------------------

# Ask user for the secret bitstring (any length)
while True:
    s = input("Enter the secret bitstring (e.g., 1011): ")
    if len(s) > 0 and all(bit in '01' for bit in s):
        break
    print("Invalid input. Please enter a non-empty string of bits (0 or 1).")

# -- build and run circuit --
qc = bv_circuit(s)
simulator = AerSimulator()
result = simulator.run(qc, shots=1000).result()
counts = result.get_counts(qc)

# -- display --
print("Measurement result:", counts)
circuit_drawer(qc, output="mpl")
plot_histogram(counts)
